**TRANSFORMACIONES - files/World_Happiness_Report.csv**

*SIN IMPUTAR - solo nuevas columnas de % y binario (yes/no)*

__________

**Where is this data set from?**

- The World Happiness Report is an annual publication of the United Nations Sustainable Development Solutions Network. This dataset is a subset of the larger report, which includes data from various sources such as the Gallup World Poll and other national surveys. The data was extracted from the World Happiness Report and made available for public use. However, the original data was collected by various researchers and organizations as part of their ongoing efforts to measure and understand happiness and well-being around the world.

    We use observed data on the six variables and estimates of their associations with life evaluations to explain the variation across countries. They include GDP per capita, social support, healthy life expectancy, freedom, generosity, and corruption. Our happiness rankings are not based on any index of these six factors – the scores are instead based on individuals’ own assessments of their lives, in particular, their answers to the single-item Cantril ladder life-evaluation question, much as epidemiologists estimate the extent to which life expectancy is affected by factors such as smoking, exercise, and diet

Detailed information about each of the Predictors:

1. **Log GDP per capita** is in terms of Purchasing Power Parity (PPP) adjusted to a constant 2017 international dollars, taken from the World Development Indicators (WDI) by the World Bank (version 17, metadata last updated on January 22, 2023). See Statistical Appendix 1 for more details. GDP data for 2022 are not yet available, so we extend the GDP time series from 2021 to 2022 using country-specific forecasts of real GDP growth from the OECD Economic Outlook No. 112 (November 2022) or, if missing, from the World Bank’s Global Economic Prospects (last updated: January 10, 2023), after adjustment for population growth. The equation uses the natural log of GDP per capita, as this form fits the data significantly better than GDP per capita.

2. The time series for **Healthy life expectancy at birth** is constructed based on data from the World Health Organization (WHO) Global Health Observatory data repository, with data available for 2005, 2010, 2015, 2016, and 2019. To match this report’s sample period (2005-2022), interpolation and extrapolation are used. See Statistical Appendix 1 for more details.

3. **Social support** - *Conversion: % y yes/no*

    **Social support** (0-1) is the national average of the binary responses (0=no, 1=yes) to the Gallup World Poll (GWP) question “If you were in trouble, do you have relatives or friends you can count on to help you whenever you need them, or not?”

4.  **Freedom to make life choices** - *Conversion: % y yes/no* 

    **Freedom to make life choices** (0-1) is the national average of binary responses to the GWP question “Are you satisfied or dissatisfied with your freedom to choose what you do with your life?”

5. **Generosity** is the residual of regressing the national average of GWP responses to the donation question “Have you donated money to a charity in the past month?” on log GDP per capita.

6.  **Perceptions of corruption** - *Conversion: % y yes/no*  

    **Perceptions of corruption** (0-1) are the average of binary answers to two GWP questions: “Is corruption widespread throughout the government or not?” and “Is corruption widespread within businesses or not?” Where data for government corruption are missing, the perception of business corruption is used as the overall corruption perception measure.

7. **Positive affect** - *Conversion: % y yes/no* 
**Positive affect** is defined as the average of previous-day effects measures for laughter, enjoyment, and interest. The inclusion of interest (first added for World Happiness Report 2022), gives us three components in each of positive and negative affect, and slightly improves the equation fit in column 4. The general form for the affect questions is: Did you experience the following feelings during a lot of the day yesterday?

8. **Negative affect** - *Conversion: % y yes/no* 
**Negative affect** is defined as the average of previous-day effects measures for worry, sadness, and anger.

9. **Life ladder**: Life evaluations from the Gallup World Poll provide the basis for the annual happiness rankings. They are based on answers to the main life evaluation question. The Cantril ladder asks respondents to think of a ladder, with the **best possible life for them being a 10 and the worst possible life being a 0**. They are then asked to rate their own current lives **on a 0 to 10 scale**. The rankings are from nationally representative samples over three years.

10. **Confidence in National Government**: The "Confidence in National Government" variable in the World Happiness Report is calculated based on the following question asked in the Gallup World Poll:

    "Do you have confidence in the national government?"

    Respondents are given the following options to choose from:

    - "Yes, always"
    - "Yes, sometimes"
    - "No, rarely"
    - "No, never"
    - "Don't know"

    **The variable is calculated as the percentage of respondents who answer "Yes, always" or "Yes, sometimes" to this question.**

    This variable is one of several social factors that are included in the calculation of the World Happiness Report's overall happiness score for each country. The report combines data on social factors such as income, social support, life expectancy, freedom to make life choices, generosity, and perceptions of corruption to arrive at a comprehensive measure of happiness.

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest

# Scripts de soporte
# -------------------------------------------------
from src.tolookandcompare import to_doc_info, to_doc_headtail, transform_info, transform_headtail

from src import soporte_eda as sp_eda
from src.soporte_eda import resumen_df

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames


**Leer/subir el .csv**

**TRANSFORMACION de `Country Name` al subir el .csv**

In [2]:
# Se edito src --> tolookandcompare_v2.py que estaba rompiendo 'Country Name'
df = pd.read_csv('files/World_Happiness_Report.csv') 

df.sample(5)

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government
267,Bulgaria,Central and Eastern Europe,2017,5.096902,9.974589,0.941755,66.099998,0.689047,-0.158122,0.910800,0.541951,0.188637,0.326088
251,Brazil,Latin America and Caribbean,2015,6.546897,9.616577,0.906693,64.699997,0.798935,-0.018645,0.771339,0.687064,0.324699,0.198535
596,Estonia,Central and Eastern Europe,2008,5.451938,10.296265,0.903726,66.339996,0.642325,-0.221212,0.662770,0.594615,0.217813,0.330979
2072,United States,North America and ANZ,2007,7.512688,10.931180,NaN,66.760002,0.871904,0.192796,0.633035,0.755915,0.231679,0.393245
1711,Singapore,Southeast Asia,2007,6.833755,11.211714,0.920632,71.760002,0.866892,0.288682,0.063615,0.587674,0.114407,0.933893


In [3]:
df.shape

(2199, 13)

In [4]:
df['Country Name'].dtype

dtype('O')

In [5]:
# NULOS - 0
df['Country Name'].isna().sum()

np.int64(0)

In [6]:
df['Country Name'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain',
       'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Estonia', 'Eswatini', 'Ethiopia', 'Finland',
       'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana',
       'Greece', 'Guatemala', 'Guinea', 'Guyana', 'Haiti', 'Honduras',
       'Hong Kong S.A.R. of China', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Ivory Coast', 'Jamaica', 'Japan', 

In [7]:
df['Country Name'].value_counts()

Country Name
Argentina     17
Costa Rica    17
Brazil        17
Bolivia       17
Bangladesh    17
              ..
Cuba           1
Maldives       1
Guyana         1
Oman           1
Suriname       1
Name: count, Length: 165, dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2199 entries, 0 to 2198
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Country Name                       2199 non-null   object 
 1   Regional Indicator                 2087 non-null   object 
 2   Year                               2199 non-null   int64  
 3   Life Ladder                        2199 non-null   float64
 4   Log GDP Per Capita                 2179 non-null   float64
 5   Social Support                     2186 non-null   float64
 6   Healthy Life Expectancy At Birth   2145 non-null   float64
 7   Freedom To Make Life Choices       2166 non-null   float64
 8   Generosity                         2126 non-null   float64
 9   Perceptions Of Corruption          2083 non-null   float64
 10  Positive Affect                    2175 non-null   float64
 11  Negative Affect                    2183 non-null   float

_________________________

📊 Transformación del PIB:

Se ha transformado/deslogaritmado la variable `Log GDP Per Capita` para obtener un PIB per cápita más interpretable.
Además, se ha creado una métrica sencilla de felicidad relativa al nivel de riqueza.

El código es robusto e idempotente, preparado para valores nulos y ejecuciones repetidas, y los resultados se utilizan posteriormente en Power BI para facilitar el análisis y la visualización.

Columnas Nuevas: 
- `GDP` - PIB per capita *real* (sueldo anual)
- `GDP_k`- GDP dividido or 1000 (para las etiquetas 1k = 1000 / 65,000 = 65k - *mas limpio*)
- `Happiness_per_GDP` - Es una métrica relativa, no absoluta.

**Happiness_per_GDP** - Es una métrica relativa, no absoluta.

Fórmula simplificada:

felicidad ÷ riqueza

🧠 Sirve para responder:

“¿Quién es más feliz para el dinero que tiene?”

“¿Quién convierte mejor riqueza en bienestar?”

✔️ Ideal para rankings y comparaciones
❌ No usar como valor económico real

In [9]:
# 1. GDP desde log (si hay NaN, se queda NaN)
df["GDP"] = np.exp(df["Log GDP Per Capita"])

# 2. GDP en miles (evita divisiones raras)
df["GDP_k"] = df["GDP"] / 1000

# 3. Felicidad relativa al GDP
# Solo se calcula si GDP_k > 0 y Life Ladder no es nulo
df["Happiness_per_GDP"] = np.where(
    (df["GDP_k"] > 0) & (df["Life Ladder"].notna()),
    df["Life Ladder"] / df["GDP_k"],
    np.nan
)

In [10]:
df.columns

Index(['Country Name', 'Regional Indicator', 'Year', 'Life Ladder',
       'Log GDP Per Capita', 'Social Support',
       'Healthy Life Expectancy At Birth', 'Freedom To Make Life Choices',
       'Generosity', 'Perceptions Of Corruption', 'Positive Affect',
       'Negative Affect', 'Confidence In National Government', 'GDP', 'GDP_k',
       'Happiness_per_GDP'],
      dtype='object')

_________________________

**TRANSFORMACION de las Columnas (5):**

- `Social Support`
- `Freedom To Make Life Choices`
- `Perceptions Of Corruption`
- `Positive Affect`
- `Negative Affect`

Columnas Nuevas: 
- valor numerico decimal en % *(% de si)* 
- valor categorico en binario *(No = hasta .5, Yes = .5+)*, NaN categorico a *"Data not available"*
- sin imputar nulos

In [11]:
# Lista de columnas originales que queremos transformar
cols = [
    "Social Support",
    "Freedom To Make Life Choices",
    "Perceptions Of Corruption",
    "Positive Affect",
    "Negative Affect"
]

for col in cols:
    print(f"\n🔎 Procesando columna: {col}")

    # 1. Contar nulos antes de transformar
    n_nulos = df[col].isna().sum()
    print(f"   ➤ Nulos encontrados: {n_nulos}")

    # 2. Convertir el nombre a Title Case para consistencia en las columnas nuevas
    col_title = col.title()

    # 3. Crear columna en formato porcentaje
    df[f"{col_title} (%)"] = (df[col] * 100).round(1)
    print(f"   ✔ Columna creada: '{col_title} (%)'")

    # 4. Crear columna categórica Yes/No
    df[f"{col_title} (Yes/No)"] = df[col].apply(
        lambda x: "Yes" if pd.notnull(x) and x >= 0.5 else
                  "No" if pd.notnull(x) else
                  "Data not available"
    )
    print(f"   ✔ Columna creada: '{col_title} (Yes/No)'")

    # 5. Mostrar ejemplo rápido de los primeros valores transformados
    print(df[[col, f"{col_title} (%)", f"{col_title} (Yes/No)"]].head(3))




🔎 Procesando columna: Social Support
   ➤ Nulos encontrados: 13
   ✔ Columna creada: 'Social Support (%)'
   ✔ Columna creada: 'Social Support (Yes/No)'
   Social Support  Social Support (%) Social Support (Yes/No)
0        0.450662                45.1                      No
1        0.552308                55.2                     Yes
2        0.539075                53.9                     Yes

🔎 Procesando columna: Freedom To Make Life Choices
   ➤ Nulos encontrados: 33
   ✔ Columna creada: 'Freedom To Make Life Choices (%)'
   ✔ Columna creada: 'Freedom To Make Life Choices (Yes/No)'
   Freedom To Make Life Choices  Freedom To Make Life Choices (%)  \
0                      0.718114                              71.8   
1                      0.678896                              67.9   
2                      0.600127                              60.0   

  Freedom To Make Life Choices (Yes/No)  
0                                   Yes  
1                                   Yes  

*NaNs % se mantienen porque Tableau/Power BI pueden graficarla sin errores.*

In [12]:
df.sample(5)

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government,GDP,GDP_k,Happiness_per_GDP,Social Support (%),Social Support (Yes/No),Freedom To Make Life Choices (%),Freedom To Make Life Choices (Yes/No),Perceptions Of Corruption (%),Perceptions Of Corruption (Yes/No),Positive Affect (%),Positive Affect (Yes/No),Negative Affect (%),Negative Affect (Yes/No)
201,Bolivia,Latin America and Caribbean,2009,6.085579,8.756054,0.831320,61.500000,0.778939,-0.037486,0.762605,0.742099,0.372369,0.468150,6349.008418,6.349008,0.958509,83.1,Yes,77.9,Yes,76.3,Yes,74.2,Yes,37.2,No
1999,Turkmenistan,Commonwealth of Independent States,2013,5.391763,9.312131,0.845733,61.060001,0.704529,-0.071969,NaN,0.551943,0.159606,NaN,11071.515612,11.071516,0.486994,84.6,Yes,70.5,Yes,NaN,Data not available,55.2,Yes,16.0,No
1674,Senegal,Sub-Saharan Africa,2015,4.617001,8.010487,0.701535,58.200001,0.719533,-0.114150,0.765490,0.709880,0.207668,0.622599,3012.382591,3.012383,1.532674,70.2,Yes,72.0,Yes,76.5,Yes,71.0,Yes,20.8,No
767,Haiti,Latin America and Caribbean,2008,3.846329,8.012043,0.679098,17.360001,0.464971,0.216526,0.811659,0.572523,0.255774,0.236633,3017.074701,3.017075,1.274854,67.9,Yes,46.5,No,81.2,Yes,57.3,Yes,25.6,No
277,Burkina Faso,Sub-Saharan Africa,2011,4.785367,7.450128,0.709528,51.639999,0.724568,-0.103416,0.706798,0.578328,0.204736,0.632865,1720.082617,1.720083,2.782057,71.0,Yes,72.5,Yes,70.7,Yes,57.8,Yes,20.5,No


In [13]:
df['Country Name'].dtype

dtype('O')

In [14]:
# NULOS - sigue siendo 0
df['Country Name'].isna().sum()

np.int64(0)

---------------------

Anadir Columna: `Continent`

In [15]:
# -------------------------------
# Country → Continent mapping
# -------------------------------

africa = [
    "Algeria","Angola","Benin","Botswana","Burkina Faso","Burundi",
    "Central African Republic","Cameroon","Chad","Comoros","Congo (Brazzaville)",
    "Congo (Kinshasa)","Djibouti","Egypt","Eswatini","Ethiopia","Gabon",
    "Gambia","Ghana","Guinea","Ivory Coast","Kenya","Lesotho","Liberia",
    "Libya","Madagascar","Malawi","Mali","Mauritania","Mauritius",
    "Morocco","Mozambique","Namibia","Niger","Nigeria","Rwanda",
    "Senegal","Sierra Leone","Somalia","Somaliland region",
    "South Africa","South Sudan","Sudan","Tanzania","Togo","Tunisia",
    "Uganda","Zambia","Zimbabwe"
]

americas = [
    "Argentina","Bolivia","Brazil","Canada","Chile","Colombia",
    "Costa Rica","Cuba","Dominican Republic","Ecuador","El Salvador",
    "Guatemala","Guyana","Haiti","Honduras","Jamaica","Mexico",
    "Nicaragua","Panama","Paraguay","Peru","Suriname",
    "Trinidad and Tobago","United States","Uruguay","Venezuela",
    "Belize"
]

asia = [
    "Afghanistan","Armenia","Azerbaijan","Bahrain","Bangladesh","Bhutan",
    "Cambodia","China","Georgia","Hong Kong S.A.R. of China","India",
    "Indonesia","Iran","Iraq","Israel","Japan","Jordan","Kazakhstan",
    "Kuwait","Kyrgyzstan","Laos","Lebanon","Malaysia","Maldives",
    "Mongolia","Myanmar","Nepal","Oman","Pakistan","Philippines",
    "Qatar","Saudi Arabia","Singapore","South Korea","Sri Lanka",
    "State of Palestine","Syria","Taiwan Province of China","Tajikistan",
    "Thailand","Turkiye","Turkmenistan","United Arab Emirates",
    "Uzbekistan","Vietnam","Yemen"
]

europe = [
    "Albania","Austria","Belarus","Belgium","Bosnia and Herzegovina",
    "Bulgaria","Croatia", "Cyprus", "Czechia","Denmark","Estonia","Finland","France",
    "Germany","Greece","Hungary","Iceland","Ireland","Italy","Kosovo",
    "Latvia","Lithuania","Luxembourg","Malta","Moldova","Montenegro",
    "Netherlands","North Macedonia","Norway","Poland","Portugal",
    "Romania","Russia","Serbia","Slovakia","Slovenia","Spain","Sweden",
    "Switzerland","Ukraine","United Kingdom"
]

oceania = [
    "Australia","New Zealand"
]

# Build dictionary
continent_map = {}

for c in africa:
    continent_map[c] = "Africa"
for c in americas:
    continent_map[c] = "Americas"
for c in asia:
    continent_map[c] = "Asia"
for c in europe:
    continent_map[c] = "Europe"
for c in oceania:
    continent_map[c] = "Oceania"

# Apply mapping
df["Continent"] = df["Country Name"].map(continent_map)


In [16]:
# Comprobación rápida (para asegurar que ya se asignó bien)
# =========================================================
print(df["Continent"].value_counts(dropna=False))

faltan = df.loc[df["Continent"].isna(), "Country Name"].dropna().unique()
print(f"\nPaíses sin continente asignado: {len(faltan)}")
print(faltan)

Continent
Asia        636
Europe      623
Africa      543
Americas    365
Oceania      32
Name: count, dtype: int64

Países sin continente asignado: 0
[]


In [17]:
# Unique country names listed:
df["Country Name"].nunique()

165

In [18]:
# Count de paises (max 18 por pais = 2005-2022)
df["Country Name"].value_counts().sort_values()

Country Name
Suriname         1
Maldives         1
Guyana           1
Cuba             1
Oman             1
                ..
Nicaragua       17
Peru            17
Saudi Arabia    17
Vietnam         17
Venezuela       17
Name: count, Length: 165, dtype: int64

In [19]:
# Paises unicos por continente:
df.groupby("Continent")["Country Name"].nunique()

Continent
Africa      49
Americas    27
Asia        46
Europe      41
Oceania      2
Name: Country Name, dtype: int64

`Country Name` que aparecen en menos de 5 anos de los 18:

Algunos países aparecen en menos de cinco años del periodo analizado (2005–2022), lo que refleja una cobertura desigual del World Happiness Report y se tiene en cuenta al interpretar los resultados.

In [20]:
# Countries with fewer than 5 years of data
few_years = df["Country Name"].value_counts()
few_years[few_years < 5]

Country Name
Somaliland region    4
Djibouti             4
Angola               4
South Sudan          4
Gambia               4
Lesotho              4
Bhutan               3
Eswatini             3
Somalia              3
Belize               2
Cuba                 1
Maldives             1
Guyana               1
Oman                 1
Suriname             1
Name: count, dtype: int64

--------

Reorganizar orden de columnas:

In [21]:
# 1) Orden lógico para vuestro modelo (Tableau/Power BI friendly)
preferred_order = [
    # Identificadores
    "Country Name", "Regional Indicator", "Continent", "Year",

    # Variables objetivo / base
    "Life Ladder",

    # Economía
    "Log GDP Per Capita", "GDP", "GDP_k", "Happiness_per_GDP",

    # Factores principales
    "Social Support", "Healthy Life Expectancy At Birth", "Freedom To Make Life Choices",
    "Generosity", "Perceptions Of Corruption",

    # Afectos
    "Positive Affect", "Negative Affect",

    # Confianza
    "Confidence In National Government",

    # Versiones en % y banderas Yes/No (tal cual vuestro CSV)
    "Social Support (%)", "Social Support (Yes/No)",
    "Freedom To Make Life Choices (%)", "Freedom To Make Life Choices (Yes/No)",
    "Perceptions Of Corruption (%)", "Perceptions Of Corruption (Yes/No)",
    "Positive Affect (%)", "Positive Affect (Yes/No)",
    "Negative Affect (%)", "Negative Affect (Yes/No)"
]

# 2) Reordenar SIN romper si falta alguna (robusto)
ordered_cols = [c for c in preferred_order if c in df.columns]
remaining_cols = [c for c in df.columns if c not in ordered_cols]
df = df[ordered_cols + remaining_cols]

In [22]:
# 3) Ver qué países faltan por mapear (para completar continent_map)
missing = df.loc[df["Continent"].isna(), "Country Name"].dropna().unique()
print(f"Países sin continente asignado: {len(missing)}")
print(missing)

Países sin continente asignado: 0
[]


*NOTA: `Regional Indicator` no se debe imputar ni rellenar aunque hay 112 NaN* 

es la metadata de WHR y ChatGPT me niega solucionaro aunque no lo entiendo, recomiendo usar `Continente` y dejar los `Nan` en `Regional Indicator`

"No se imputa la variable Regional Indicator porque contiene valores ausentes reales del conjunto de datos original y completarla implicaría introducir supuestos no respaldados por la fuente."

"No se imputa Regional Indicator porque es una variable categórica editorial del WHR con valores ausentes reales; completarla implicaría introducir supuestos no respaldados por la fuente." - ChatGPT

In [23]:
df["Regional Indicator"].value_counts(dropna=False)

Regional Indicator
Sub-Saharan Africa                    443
Latin America and Caribbean           321
Western Europe                        303
Central and Eastern Europe            241
Middle East and North Africa          204
Commonwealth of Independent States    188
Southeast Asia                        132
NaN                                   112
South Asia                             97
East Asia                              92
North America and ANZ                  66
Name: count, dtype: int64

In [24]:
df["Regional Indicator"].isnull().sum()

np.int64(112)

____

#### **GUARDAR nuevo .csv "World Happiness Report limpio sin imputar":** 

In [25]:
df.to_csv('files/WHR_limpio_sin_imputar.csv', index=False)

In [26]:
df.head(5)

,Country Name,Regional Indicator,Continent,Year,Life Ladder,Log GDP Per Capita,GDP,GDP_k,Happiness_per_GDP,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government,Social Support (%),Social Support (Yes/No),Freedom To Make Life Choices (%),Freedom To Make Life Choices (Yes/No),Perceptions Of Corruption (%),Perceptions Of Corruption (Yes/No),Positive Affect (%),Positive Affect (Yes/No),Negative Affect (%),Negative Affect (Yes/No)
0,Afghanistan,South Asia,Asia,2008,3.723590,7.350416,1556.844325,1.556844,2.391755,0.450662,50.500000,0.718114,0.167652,0.881686,0.414297,0.258195,0.612072,45.1,No,71.8,Yes,88.2,Yes,41.4,No,25.8,No
1,Afghanistan,South Asia,Asia,2009,4.401778,7.508646,1823.742543,1.823743,2.413596,0.552308,50.799999,0.678896,0.190809,0.850035,0.481421,0.237092,0.611545,55.2,Yes,67.9,Yes,85.0,Yes,48.1,No,23.7,No
2,Afghanistan,South Asia,Asia,2010,4.758381,7.613900,2026.164160,2.026164,2.348468,0.539075,51.099998,0.600127,0.121316,0.706766,0.516907,0.275324,0.299357,53.9,Yes,60.0,Yes,70.7,Yes,51.7,Yes,27.5,No
3,Afghanistan,South Asia,Asia,2011,3.831719,7.581259,1961.095989,1.961096,1.953866,0.521104,51.400002,0.495901,0.163571,0.731109,0.479835,0.267175,0.307386,52.1,Yes,49.6,No,73.1,Yes,48.0,No,26.7,No
4,Afghanistan,South Asia,Asia,2012,3.782938,7.660506,2122.830826,2.122831,1.782025,0.520637,51.700001,0.530935,0.237588,0.775620,0.613513,0.267919,0.435440,52.1,Yes,53.1,Yes,77.6,Yes,61.4,Yes,26.8,No
